<a href="https://colab.research.google.com/github/NATASHASAINI/mongodb/blob/main/Copy_of_assignmentchatbotragnatasha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets pandas pymongo sentence_transformers
!pip install -U sentence_transformers
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 27.1 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset#Implement the process of chunking your knowledge base documents.
import pandas as pd
dataset = load_dataset("Voxel51/fiftyone-embeddings-combined")
dataset_df = pd.DataFrame(dataset['train'])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/254M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/252M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/28118 [00:00<?, ? examples/s]

In [4]:
#Removing the preexisting embedding to generate own embeddings
print(dataset_df.isnull().sum())
dataset_df = dataset_df.drop(columns=['embedding'])

query               0
response           83
embedding           0
embedding_model     0
source_dataset      0
content_type        0
dataset_type        0
dtype: int64


In [5]:
#Generate own embeddings
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
def get_embedding(text:str) -> list[float]:
  if not text.strip():
    print("attempted to get embedding for every step")
    return[]
  embedding =embedding_model.encode(text)
  return embedding.tolist()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
!pip install pymongo

In [7]:
from requests import get
ip = get('https://api.ipify.org').text
print(f"Your Colab IP: {ip}")


Your Colab IP: 34.83.14.213


In [8]:
import pymongo
from google.colab import userdata

def get_mongo_client(mongo_uri):
    """Establish the connection to MongoDB"""
    try:
        # Append tlsAllowInvalidCertificates=true to handle SSL handshake issues
        if "?" in mongo_uri:
            mongo_uri += "&tlsAllowInvalidCertificates=true"
        else:
            mongo_uri += "?tlsAllowInvalidCertificates=true"

        client = pymongo.MongoClient(mongo_uri)
        # The ismaster command is cheap and does not require auth.
        client.admin.command('ismaster')
        print("Connection to MongoDB successful")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None

mongo_uri = userdata.get("mongo_uri")
# Clean the mongo_uri to ensure only a single URI is used
if mongo_uri and "\r\n" in mongo_uri:
    mongo_uri = mongo_uri.split("\r\n")[0]

if not mongo_uri:
   print("MONGO_URI not set an env variable")
mongo_client = get_mongo_client(mongo_uri)

#Ingest these embeddings into your chosen vector store
db = mongo_client["chatbot"]
collection = db["embeddings"]

Connection to MongoDB successful


In [9]:
collection.delete_many({})

DeleteResult({'n': 0, 'electionId': ObjectId('7fffffff00000000000001b2'), 'opTime': {'ts': Timestamp(1769369688, 17), 't': 434}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1769369688, 17), 'signature': {'hash': b'f\xc8\xa1%0\xfa\xccb\xe1$K\x12\x97v \xb6\xd6\x06\xde\xf0', 'keyId': 7563759483558559746}}, 'operationTime': Timestamp(1769369688, 17)}, acknowledged=True)

In [10]:
documents = dataset_df.to_dict('records')
collection.insert_many(documents)
print("Data Ingestion to MongoDB inserted successfully")

Data Ingestion to MongoDB inserted successfully
